In [6]:
import os
import sys
import numpy as np
import pandas as pd
import time

In [14]:
def read_collection(file_collection):
    """Function that reads tab seperated gzip collection file"""
    print ( "START: reading CSV file PRIM..." )
    start_time = time.time()
    f_tab = open(file_collection, 'r')
    df_collection = pd.read_csv(f_tab, index_col=False, header=0, delimiter="\t", compression="gzip") # index is snpID. # production_v2 - NEW March 2015. *REMEMBER TO correct "locate_collection_file()" as well*
    f_tab.close()
    elapsed_time = time.time() - start_time
    print( "END: read CSV file PRIM into DataFrame in %s s (%s min)" % (elapsed_time, elapsed_time/60) )
    return df_collection

In [15]:
file_collection = "/Users/djw472/data/locus_definition/SNPsnap_collection_downloads/EUR_1KG_phase3-ld0.5_collection.tab.gz"
df_collection = read_collection(file_collection)

START: reading CSV file PRIM...
END: read CSV file PRIM into DataFrame in 95.0448288918 s (1.58408048153 min)


In [16]:
df_collection.head()

,snpID,rsID,freq_bin,snp_maf,gene_count,dist_nearest_gene_snpsnap,dist_nearest_gene_snpsnap_protein_coding,dist_nearest_gene,dist_nearest_gene_located_within,loci_upstream,...,ID_genes_in_matched_locus,friends_ld01,friends_ld02,friends_ld03,friends_ld04,friends_ld05,friends_ld06,friends_ld07,friends_ld08,friends_ld09
0,10:10001753,10:10001753,7,0.07455,0,98932.0,1045506.0,98932.0,inf,9986766,...,NaN,253.0,103.0,55.0,40.0,35.0,33.0,31.0,20.0,0.0
1,10:10001794,10:10001794,41,0.41050,0,98891.0,1045465.0,98891.0,inf,9964948,...,NaN,365.0,299.0,294.0,266.0,168.0,138.0,58.0,45.0,0.0
2,10:100023489,10:100023489,10,0.10540,1,4518.0,4518.0,4518.0,4518.000000,100023489,...,ENSG00000138131,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10:100025128,10:100025128,45,0.45430,1,2879.0,2879.0,2879.0,2879.000000,100025128,...,ENSG00000138131,112.0,70.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10:10002975,10:10002975,1,0.01193,2,97710.0,1044284.0,97710.0,inf,9691526,...,ENSG00000235838;ENSG00000228636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### Check for uniqueness
n_unique_rsid = len(df_collection["rsID"].unique())
n_rsid = len(df_collection)

print n_unique_rsid
print n_rsid
print n_unique_rsid == n_rsid

9535059
9535059
True


## Export

In [19]:
df_collection_red = df_collection[["snpID", "rsID", "snp_maf"]] # produces a copy
df_collection_red.head()

,snpID,rsID,snp_maf
0,10:10001753,10:10001753,0.07455
1,10:10001794,10:10001794,0.41050
2,10:100023489,10:100023489,0.10540
3,10:100025128,10:100025128,0.45430
4,10:10002975,10:10002975,0.01193


In [24]:
tmp_series = df_collection_red["snpID"].str.split(':') # split return a Series of lists
tmp_series
df_collection_red["chr"] = tmp_series.str.get(0)
df_collection_red["pos"] = tmp_series.str.get(1)

/Users/djw472/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/djw472/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [25]:
df_collection_red.head()

,snpID,rsID,snp_maf,chr,pos
0,10:10001753,10:10001753,0.07455,10,10001753
1,10:10001794,10:10001794,0.41050,10,10001794
2,10:100023489,10:100023489,0.10540,10,100023489
3,10:100025128,10:100025128,0.45430,10,100025128
4,10:10002975,10:10002975,0.01193,10,10002975


In [27]:
df_collection_export = df_collection_red.drop(columns="snpID")
df_collection_export.head()

,rsID,snp_maf,chr,pos
0,10:10001753,0.07455,10,10001753
1,10:10001794,0.41050,10,10001794
2,10:100023489,0.10540,10,100023489
3,10:100025128,0.45430,10,100025128
4,10:10002975,0.01193,10,10002975


In [28]:
### Write tab file
df_collection_export.to_csv("snpsnap_EUR_1KG_phase3-chrpos_mapping.tab.gz", sep="\t", index=False, compression='gzip')